#### Duration: nearly instant

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from pathlib import Path
import sys
import logging
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/bin/python


In [2]:
from multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D import Inferencer2D

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Python interpreter binary location: /n/groups/datta/tim_sainburg/conda_envs/mmdeploy/bin/python


### Get recording info (google sheets)

In [3]:
import requests
import pandas as pd
from io import BytesIO

In [18]:
spreadsheet_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vROizjMMFZBEjxaWMiGj1T1veMFwTOxbsTC7xttf4tLZXbA5npUfUCDYu5gCNMOwIo-nkwicDv8W-B7/pub?gid=2042187371&single=true&output=csv"
# spreadsheet_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vROizjMMFZBEjxaWMiGj1T1veMFwTOxbsTC7xttf4tLZXbA5npUfUCDYu5gCNMOwIo-nkwicDv8W-B7/pubhtml"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [19]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m,use_local,User
0,V2C-2-l,60,24-06-07-12-37-37-680000,NaN,20240607_calibration,/n/groups/datta/byron/theta/experiment3_septum...,NaN,/n/groups/datta/byron/theta/experiment3_septum...,120,byp166,0,120,False,NaN
1,V2C-2-lr,60,24-06-07-14-20-44-808560,NaN,20240607_calibration,/n/groups/datta/byron/theta/experiment3_septum...,NaN,/n/groups/datta/byron/theta/experiment3_septum...,120,byp166,0,120,False,NaN
2,V2C-2-l,60,24-06-21-10-43-59-959340,NaN,20240621_calibration,/n/groups/datta/byron/theta/experiment3_septum...,NaN,/n/groups/datta/byron/theta/experiment3_septum...,120,byp166,0,120,False,NaN


### Run sync

In [20]:
for idx, recording_row in recording_df.iterrows():
    break

In [21]:
print(recording_row)

Subject                                                                 V2C-2-l
duration_m                                                                   60
video_recording_id                                     24-06-07-12-37-37-680000
ephys_id                                                                    NaN
calibration_id                                             20240607_calibration
video_location_on_o2          /n/groups/datta/byron/theta/experiment3_septum...
ephys_location_on_o2                                                        NaN
calibration_location_on_o2    /n/groups/datta/byron/theta/experiment3_septum...
samplerate                                                                  120
username                                                                 byp166
n_ephys_streams                                                               0
max_video_duration_m                                                        120
use_local                               

In [22]:
SAMPLERATE = 120
TRIGGER_PIN = 2

In [23]:
output_directory = Path("/n/groups/datta/byron/ephys_pipeline/scratch") / "septumopto-3d-pipeline"
print(output_directory)

/n/groups/datta/byron/ephys_pipeline/scratch/septumopto-3d-pipeline


In [24]:
output_directory_camera_sync = output_directory / 'camera_sync' / recording_row.video_recording_id
output_directory_camera_sync.mkdir(parents=True, exist_ok=True)

In [25]:
from multicamera_airflow_pipeline.tim_240731.sync.sync_cameras import CameraSynchronizer

In [26]:
dir_name = recording_row.video_location_on_o2 + "/" + recording_row.video_recording_id
video_directory = Path(dir_name)

In [27]:
help(CameraSynchronizer)

Help on class CameraSynchronizer in module multicamera_airflow_pipeline.tim_240731.sync.sync_cameras:

class CameraSynchronizer(builtins.object)
 |  CameraSynchronizer(recording_directory, output_directory, samplerate, trigger_pin=2, recompute_completed=False)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, recording_directory, output_directory, samplerate, trigger_pin=2, recompute_completed=False)
 |      Parameters
 |      ----------
 |      recording_directory : Path
 |          Path to the directory containing the recording data.
 |      output_location : Path
 |          Path to the directory where the synchronized data will be saved.
 |      trigger_pin : int
 |          GPIO pin number used to trigger the recording.
 |  
 |  check_completed(self)
 |  
 |  check_if_correct_fps(self)
 |  
 |  load_config(self)
 |  
 |  load_metadata(self)
 |  
 |  load_triggerdata(self)
 |  
 |  run(self)
 |  
 |  ----------------------------------------------------------------------
 |  Da

In [28]:
synchronizer = CameraSynchronizer(
    recording_directory = video_directory,
    output_directory = output_directory_camera_sync,
    samplerate = SAMPLERATE, # camera sample rate
    trigger_pin= 2, # Which pin camera trigger was on
)

In [29]:
synchronizer.run()

INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras:Checking if correct FPS
INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras:Loading video config, metadata, and triggerdata
INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras:	 aligning frames for camera: bottom


ValueError: could not convert string 'None' to float64 at row 0, column 5.

In [27]:
!ls {output_directory_camera_sync}

camera_sync.csv


In [30]:
!head {output_directory_camera_sync/'camera_sync.csv'}

,trigger_times,trigger_states,BackBottom_frames,BackLeft_frames,BackRight_frames,FrontBottom_frames,FrontLeft_frames,FrontRight_frames,datetime_est
0,0.0,0,0,0,0,0,0,0,1714584403.110846
1,6666.0,0,1,1,1,1,1,1,1714584403.117512
2,13332.0,1,2,2,2,2,2,2,1714584403.124178
3,19998.0,1,3,3,3,3,3,3,1714584403.130844
4,26664.0,1,4,4,4,4,4,4,1714584403.13751
5,33330.0,0,5,5,5,5,5,5,1714584403.144176
6,39996.0,0,6,6,6,6,6,6,1714584403.150842
7,46662.0,0,7,7,7,7,7,7,1714584403.157508
8,53328.0,0,8,8,8,8,8,8,1714584403.164174


In [29]:
pd.read_csv(output_directory_camera_sync/'camera_sync.csv')[:3]

,Unnamed: 0,trigger_times,trigger_states,BackBottom_frames,BackLeft_frames,BackRight_frames,FrontBottom_frames,FrontLeft_frames,FrontRight_frames,datetime_est
0,0,0.0,0,0,0,0,0,0,0,1.714584e+09
1,1,6666.0,0,1,1,1,1,1,1,1.714584e+09
2,2,13332.0,1,2,2,2,2,2,2,1.714584e+09
